# Sticky Key Like Backdoor Usage
Detects the usage and installation of a backdoor that uses an option to register a malicious debugger for built-in tools that are accessible in the login screen

## Rule Content
```
- action: global
  title: Sticky Key Like Backdoor Usage
  id: baca5663-583c-45f9-b5dc-ea96a22ce542
  description: Detects the usage and installation of a backdoor that uses an option
    to register a malicious debugger for built-in tools that are accessible in the
    login screen
  references:
  - https://blogs.technet.microsoft.com/jonathantrull/2016/10/03/detecting-sticky-key-backdoors/
  tags:
  - attack.privilege_escalation
  - attack.persistence
  - attack.t1015
  - car.2014-11-003
  - car.2014-11-008
  author: Florian Roth, @twjackomo
  date: 2018/03/15
  detection:
    condition: 1 of them
  falsepositives:
  - Unlikely
  level: critical
- logsource:
    product: windows
    service: sysmon
  detection:
    selection_registry:
      EventID: 13
      TargetObject:
      - '*\SOFTWARE\Microsoft\Windows NT\CurrentVersion\Image File Execution Options\sethc.exe\Debugger'
      - '*\SOFTWARE\Microsoft\Windows NT\CurrentVersion\Image File Execution Options\utilman.exe\Debugger'
      - '*\SOFTWARE\Microsoft\Windows NT\CurrentVersion\Image File Execution Options\osk.exe\Debugger'
      - '*\SOFTWARE\Microsoft\Windows NT\CurrentVersion\Image File Execution Options\Magnify.exe\Debugger'
      - '*\SOFTWARE\Microsoft\Windows NT\CurrentVersion\Image File Execution Options\Narrator.exe\Debugger'
      - '*\SOFTWARE\Microsoft\Windows NT\CurrentVersion\Image File Execution Options\DisplaySwitch.exe\Debugger'
      EventType: SetValue
- logsource:
    category: process_creation
    product: windows
  detection:
    selection_process:
      ParentImage:
      - '*\winlogon.exe'
      CommandLine:
      - '*cmd.exe sethc.exe *'
      - '*cmd.exe utilman.exe *'
      - '*cmd.exe osk.exe *'
      - '*cmd.exe Magnify.exe *'
      - '*cmd.exe Narrator.exe *'
      - '*cmd.exe DisplaySwitch.exe *'

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"13" AND registry_key_path.keyword:(*\\SOFTWARE\\Microsoft\\Windows\ NT\\CurrentVersion\\Image\ File\ Execution\ Options\\sethc.exe\\Debugger OR *\\SOFTWARE\\Microsoft\\Windows\ NT\\CurrentVersion\\Image\ File\ Execution\ Options\\utilman.exe\\Debugger OR *\\SOFTWARE\\Microsoft\\Windows\ NT\\CurrentVersion\\Image\ File\ Execution\ Options\\osk.exe\\Debugger OR *\\SOFTWARE\\Microsoft\\Windows\ NT\\CurrentVersion\\Image\ File\ Execution\ Options\\Magnify.exe\\Debugger OR *\\SOFTWARE\\Microsoft\\Windows\ NT\\CurrentVersion\\Image\ File\ Execution\ Options\\Narrator.exe\\Debugger OR *\\SOFTWARE\\Microsoft\\Windows\ NT\\CurrentVersion\\Image\ File\ Execution\ Options\\DisplaySwitch.exe\\Debugger) AND event_type:"SetValue")')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

In [ ]:
s = searchContext.query('query_string', query='(process_parent_path.keyword:(*\\winlogon.exe) AND process_command_line.keyword:(*cmd.exe\ sethc.exe\ * OR *cmd.exe\ utilman.exe\ * OR *cmd.exe\ osk.exe\ * OR *cmd.exe\ Magnify.exe\ * OR *cmd.exe\ Narrator.exe\ * OR *cmd.exe\ DisplaySwitch.exe\ *))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()